In [1]:
from findmycells.main import Project
from findmycells.preprocessing import CropStitchingArtefacts
from findmycells.segmentation import Deepflash2BinaryAndInstanceSegmentationStrategy

In [2]:
project_root_dir = '/mnt/c/Users/dsege/TEMP/test_project/'

In [17]:
import os
import shutil

os.mkdir('/mnt/c/Users/dsege/TEMP/test_project/just_for_temporary_copies/')

In [18]:
filepath_src = '/mnt/c/Users/dsege/TEMP/test_project/02_preprocessed_images/0000-000.png'
dir_dst = '/mnt/c/Users/dsege/TEMP/test_project/just_for_temporary_copies/'


shutil.copy(filepath_src, dir_dst)

'/mnt/c/Users/dsege/TEMP/test_project/just_for_temporary_copies/0000-000.png'

### Option A: start a new project

In [3]:
user_input_via_gui = {'project_root_dir': project_root_dir,
                      'preprocessing_configs': {0: {'method_category': None,
                                                    'method_specifier': None,
                                                    'additional_settings': dict(),
                                                    'ProcessingMethod': CropStitchingArtefacts(),
                                                    'ProcessingStrategy': None}
                                               },
                     'segmentation_strategy': Deepflash2BinaryAndInstanceSegmentationStrategy()}

In [4]:
test_project = Project(user_input_via_gui)

In [5]:
test_project.database.segmentation_strategy

In [6]:
file_ids = ['0000']
test_project.preprocess(file_ids)

done with plane 000
done with plane 001
done with plane 002
done with plane 003
done with plane 004
done with plane 005
done with plane 006
done with plane 007


In [7]:
test_project.save_status()

In [8]:
test_project.run_segmentation()

Found 5 models in folder /mnt/c/Users/dsege/TEMP/test_project/03_deepflash2/trained_models
['Unet_resnet34_2classes-fold1.pth', 'Unet_resnet34_2classes-fold2.pth', 'Unet_resnet34_2classes-fold3.pth', 'Unet_resnet34_2classes-fold4.pth', 'Unet_resnet34_2classes-fold5.pth']
Computing Stats...


Predicting 0000-000.png


Predicting 0000-001.png


Predicting 0000-002.png


Predicting 0000-003.png


Predicting 0000-004.png


Predicting 0000-005.png


Predicting 0000-006.png


Predicting 0000-007.png


2021-12-30 21:05:12,157 [INFO] WRITING LOG OUTPUT TO /home/ds/.cellpose/run.log
Found 5 models in folder /mnt/c/Users/dsege/TEMP/test_project/03_deepflash2/trained_models
['Unet_resnet34_2classes-fold1.pth', 'Unet_resnet34_2classes-fold2.pth', 'Unet_resnet34_2classes-fold3.pth', 'Unet_resnet34_2classes-fold4.pth', 'Unet_resnet34_2classes-fold5.pth']
Computing Stats...
Using diameter of 50
2021-12-30 21:05:59,380 [INFO] ** TORCH CUDA version installed and working. **
2021-12-30 21:05:59,383 [INFO] >>>> using GPU
2021-12-30 21:05:59,581 [INFO] ~~~ FINDING MASKS ~~~
2021-12-30 21:06:55,584 [INFO] >>>> TOTAL TIME 56.00 sec
Found 5 models in folder /mnt/c/Users/dsege/TEMP/test_project/03_deepflash2/trained_models
['Unet_resnet34_2classes-fold1.pth', 'Unet_resnet34_2classes-fold2.pth', 'Unet_resnet34_2classes-fold3.pth', 'Unet_resnet34_2classes-fold4.pth', 'Unet_resnet34_2classes-fold5.pth']
Computing Stats...
Using diameter of 50
2021-12-30 21:07:33,135 [INFO] ** TORCH CUDA version installe

In [9]:
test_project.save_status()

In [13]:
test_project.database.segmented_file_lists

{'binary_segmented_files': ['0000-000.png',
  '0000-001.png',
  '0000-002.png',
  '0000-003.png',
  '0000-004.png',
  '0000-005.png',
  '0000-006.png',
  '0000-007.png'],
 'instance_segmented_files': ['0000-000.png',
  '0000-001.png',
  '0000-002.png',
  '0000-003.png',
  '0000-004.png',
  '0000-005.png',
  '0000-006.png',
  '0000-007.png']}

### Option B: load and continue an existing project

In [ ]:
user_input_via_gui_load_from_configs = {'project_root_dir': project_root_dir}

In [ ]:
test_project = Project(user_input_via_gui_load_from_configs)

In [ ]:
test_project.load_status()

In [ ]:
file_ids = ['0002']
test_project.preprocess(file_ids)

In [ ]:
test_project.run_segmentation()

In [ ]:
test_project.save_status()

### Option C: Final test:

In [ ]:
user_input_via_gui_load_from_configs = {'project_root_dir': project_root_dir}

In [ ]:
test_project = Project(user_input_via_gui_load_from_configs)

In [ ]:
test_project.load_status()

In [ ]:
test_project.database.file_infos['preprocessing_completed']

In [ ]:
test_project.preprocess()

In [ ]:
test_project.save_status()

# This is where we want to go:

In [ ]:
## Initialize the project

project_name = Project(project_root_dir)
# project_root_dir: path of project root directory

project_name.initialize_database()
# creates the database object and sets-up everything (main subdirs, file infos, ..)
# as alternative, see: load_status()

project_name.save_current_status()
# saves current progress status to disk (basically the database.file_infos as pickle file? - what about shapely polygons that might have been loaded?)


project_name.load_status()
# Alternative to initialize_database()
# loads the corresponding information from disk and allows the user to continue from here


## Preprocessing

project_name.preprocess(microscopy_files, cropping_strategy)
# microscopy_files: list of all microscopy image files that shall be preprocessed (this would allow step-by-step processing)
# cropping_strategy: object of type CroppingStrategy that specifies which exact cropping strategy shall be used

# infers:
    # microscopy-image-file-loader from filetype (e.g. .czi file loader)
    # whether it´s a z-stack
    # whether it´s an RGB image
# automatically updates the database accordingly


## Image segmentation

project_name.run_segmentations(files, SegmentationStrategy)
# files: list of files that shall be segmented (in case of df2 make sure that all are used due to image statistics & cellpose diameter)
# SegmentationStrategy: e.g. deepflash2_binary, deepflash2_instances, intellesis

# launches the respective segmentations and saves all progress in the database object & all segmentations in the corresponding subdir


## Quantifications

project_name.run_quantifications(files, QuantificationStrategy)
# files: list of files whose segmentations shall be used for quantification
# QuantificationStrategy: e.g. 2D, 3D_grid_estimation, 3D_entire_volume (enable selection of more specific options / exclusion criteria)

# Launches the respective quantifications and saves all progress in the database object & all quantification results in the results table